In [252]:
import mysql.connector as db
import pandas as pd
import shutil
from textblob import TextBlob

#avoid repeating the values
user  = 'Sakthi'
passcode = "Sakthi@2001"
host = 'localhost'
db_name = "cust_behav_analys"
#csv files 
MSTproducts = r"C:\Users\sakth\OneDrive\Desktop\Data Science Couse\Customer Behavior Analysis\products.csv"
MSTcustomers = r"C:\Users\sakth\OneDrive\Desktop\Data Science Couse\Customer Behavior Analysis\customers.csv"
MSTgeography = r"C:\Users\sakth\OneDrive\Desktop\Data Science Couse\Customer Behavior Analysis\geography.csv"
TRNcustomer_journey = r"C:\Users\sakth\OneDrive\Desktop\Data Science Couse\Customer Behavior Analysis\customer_journey.csv"
TRNcustomer_reviews = r"C:\Users\sakth\OneDrive\Desktop\Data Science Couse\Customer Behavior Analysis\customer_reviews.csv"
TRNengagement_data = r"C:\Users\sakth\OneDrive\Desktop\Data Science Couse\Customer Behavior Analysis\engagement_data.csv"


#dynamic function for creating connection to MYSQL
def get_connection(host, user, password):
    try:
        conn = db.connect(host=host, user=user, password=password)
        print(" Connection to MySQL server successful!")
        return conn
    except db.Error as e:
        print(f" Connection Failed: {e}")
        #no need to return None python by default return none
        return None

#dynamic function for clearn Data in given csv file
def Cleaning_Data(parm):
    shutil.copy(parm, parm + '.backup')  
    result = pd.read_csv(parm)
    result.drop_duplicates(inplace=True)
        # Handle empty values based on data types
    for col in result.columns:
        if result[col].dtype == 'object':  # String columns
            empty_value = ""
        else:  # Numeric columns (int or float)
            empty_value = 0
        result[col].fillna(empty_value, inplace=True)
    return result

def spliting_Data(csv_file, Combinedcol, splitCol1, splitCol2):
    # Read CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    
    if Combinedcol in df.columns:

        # Split the specified combined column into two new columns
        df[[splitCol1, splitCol2]] = df[Combinedcol].str.split('-', expand=True)
        
        # Convert the new columns to numeric types inplace
        df[splitCol1] = pd.to_numeric(df[splitCol1], errors='coerce')
        df[splitCol2] = pd.to_numeric(df[splitCol2], errors='coerce')
        
        # Drop the original combined column inplace
        df.drop(columns=[Combinedcol], inplace=True)
        
        # Save the updated DataFrame back to the same CSV file
        df.to_csv(csv_file, index=False)
        print(f"File updated successfully: {csv_file}")
    else:
        print(f"Column '{Combinedcol}' does not exist in the file.")
    
    # Return the updated DataFrame with the new split columns included
    return df
 
#dynamic Function to create and select database
def create_database(conn, db_name):
    try:
         with conn.cursor() as cursor:
            cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name};")
            conn.database = db_name
            print(f" Database '{db_name}' created and selected successfully!")
            #The cursor is automatically closed only if we use the "with" statement.
            # cursor.close()
            #If we forget to close it, the cursor stays open, consuming memory(temp file), and could cause performance issues.
    except db.Error as e:
        print(f" Database Creation Failed: {e}")
        #Not necessary to return None. python by default return none
        return None

# Function to Create Table Dynamically
def create_table_from_csv(conn, csv_file, table_name, primary_key=None):
    # Read CSV to DataFrame
    df = pd.read_csv(csv_file)
    
    # Generate CREATE TABLE statement dynamically
    # Local variable
    columns = []
    for col, dtype in zip(df.columns, df.dtypes):
        # Map Pandas dtypes to MySQL data types
        if "date".upper() in str(col).upper():
            sql_type = "DATETIME"
            try:              
                df[col] = pd.to_datetime(df[col])
                df.to_csv(csv_file, index=False)
                print(f"File updated successfully: {csv_file}")
            except Exception as e:
                sql_type = "VARCHAR(255)"
        elif "int" in str(dtype):
            sql_type = "INT"
        elif "float" in str(dtype):
            sql_type = "FLOAT"
        else:
            sql_type = "VARCHAR(255)"   # Default for string or object types
        
        # Add column definition
        if primary_key and col == primary_key:
            columns.append(f"`{col}` {sql_type} PRIMARY KEY")
        else:
            columns.append(f"`{col}` {sql_type}")

    # Combine all columns for SQL
    columns_sql = ", ".join(columns)
    create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_sql});"
    
    try:
        with conn.cursor() as cursor:
            cursor.execute(create_table_query)
            print(f" Table '{table_name}' created successfully!")             
            insert_csv_to_table(conn, csv_file, table_name, primary_key)
    except db.Error as e:
        print(f" Failed to create table '{table_name}': {e}")

# Function to Insert CSV Data into the Table
def insert_csv_to_table(conn, csv_file, table_name, primary_key=None):
    # Data Cleaning
    df = Cleaning_Data(csv_file)

    try:
        with conn.cursor() as cursor:
            # Prepare Insert Query
            columns = ", ".join([f"`{col}`" for col in df.columns])
            values_placeholders = ", ".join(["%s"] * len(df.columns))
            
            # Check if primary_key is provided
            if primary_key and primary_key in df.columns:
                # Prepare Update Query
                update_columns = ", ".join([f"`{col}`=VALUES(`{col}`)" for col in df.columns if col != primary_key])
                insert_query = (f"INSERT INTO {table_name} ({columns}) VALUES ({values_placeholders}) "
                                f"ON DUPLICATE KEY UPDATE {update_columns}")
            else:
                # Insert only query
                insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({values_placeholders})"

            # Insert or Update Rows
            for row in df.itertuples(index=False, name=None):
                cursor.execute(insert_query, row)
            conn.commit()
            print(f"Data inserted/updated into '{table_name}' successfully!")
    #Genderal Exception        
    #except Exception as ex:   
    #Specific Exception      
    except db.Error as e:
        print(f"Failed to insert/update data: {e}")

# Function to Insert CSV Data into the Table
def convert_column(conn, table_name, column_name, to_type):

    try:
        with conn.cursor() as cursor:

            
            # Check the current data type of the column
            cursor.execute(f"SHOW COLUMNS FROM {table_name} WHERE Field = '{column_name}'")
            column_info = cursor.fetchone()

            if not column_info:
                print(f"Column '{column_name}' not found in table '{table_name}'.")
                return

            current_type = column_info[1]  # Extract data type
            
            # Proceed only if the column is of type VARCHAR
            if "varchar" in current_type.lower():
                print(f"Converting '{column_name}' from {current_type} to {to_type}...")

                # Add a temporary column of type DATE
                temp_column = f"{column_name}_temp"
                cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {temp_column} {to_type} NULL")

                # Convert and store values in the new column
                cursor.execute(f"""
                    UPDATE {table_name} 
                    SET {temp_column} = STR_TO_DATE({column_name}, '%d-%m-%Y')
                """)

                # Drop the old VARCHAR column
                cursor.execute(f"ALTER TABLE {table_name} DROP COLUMN {column_name}")

                # Rename the temporary column to original column name
                cursor.execute(f"ALTER TABLE {table_name} CHANGE COLUMN {temp_column} {column_name} {to_type}")

                print(f"Successfully converted '{column_name}' in table '{table_name}' to {to_type}.")

            else:
                print(f"Column '{column_name}' is not of type VARCHAR, it is {current_type}. No conversion needed.")

        # Commit and close connection
        conn.commit()

            
    #Genderal Exception        
    #except Exception as ex:   
    #Specific Exception      
    except db.Error as e:
        print(f"Failed to insert/update data: {e}")

# Function to execute SELECT query
def fetch_data(conn,query):
    try:        
        with conn.cursor() as cursor:
            cursor.execute(query)
            
            results = cursor.fetchall()
            
             # Fetch column names dynamically
            columns = [desc[0] for desc in cursor.description]

            # Convert results to DataFrame
            df = pd.DataFrame(results, columns=columns)
            return df
    except db.Error as e:
        print(f"Error: {e}")
    
def get_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.1:
        return "Positive"
    elif polarity < -0.1:
        return "Negative"
    else:
        return "Neutral"

# Main block
# __name__ not necessary here
# But this is necessary when import this py file in another py file
if __name__ == "__main__":
    
#Creating connection to MYSQL
    conn=get_connection(host, user, passcode)

#Creating and select db 
    create_database(conn,db_name)

#clearn Table in Mysql
    create_table_from_csv(conn,MSTproducts,"MSTproducts","ProductID")
    create_table_from_csv(conn,MSTcustomers,"MSTcustomers","CustomerID")
    create_table_from_csv(conn,MSTgeography,"MSTgeography","GeographyID")
    create_table_from_csv(conn,TRNcustomer_journey,"TRNcustomer_journey","JourneyID")
    create_table_from_csv(conn,TRNcustomer_reviews,"TRNcustomer_reviews","ReviewID")
    #spliting 'ViewsClicksCombined' column into two 'Views','Clicks'      
    spliting_Data(TRNengagement_data,'ViewsClicksCombined','Views','Clicks')
    create_table_from_csv(conn,TRNengagement_data,"TRNengagement_data","EngagementID")

#select query    
#What factors are influencing customer engagement?
    query = """
SELECT p.ProductName, COUNT(*)
FROM cust_behav_analys.TRNcustomer_reviews AS cr
JOIN cust_behav_analys.MSTproducts AS p 
ON cr.productid = p.ProductID
WHERE cr.rating BETWEEN 1 AND 3
GROUP BY cr.productid
HAVING COUNT(*) > 2
ORDER BY COUNT(*);
"""
#factors are influencing customer engagement Based on customer reviews
    fetch_data(conn,query) 


 Connection to MySQL server successful!
 Database 'cust_behav_analys' created and selected successfully!
 Table 'MSTproducts' created successfully!
Data inserted/updated into 'MSTproducts' successfully!
 Table 'MSTcustomers' created successfully!
Data inserted/updated into 'MSTcustomers' successfully!
 Table 'MSTgeography' created successfully!


C:\Users\sakth\AppData\Local\Temp\ipykernel_26172\1014440049.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result[col].fillna(empty_value, inplace=True)
C:\Users\sakth\AppData\Local\Temp\ipykernel_26172\1014440049.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

Data inserted/updated into 'MSTgeography' successfully!
File updated successfully: C:\Users\sakth\OneDrive\Desktop\Data Science Couse\Customer Behavior Analysis\customer_journey.csv
 Table 'TRNcustomer_journey' created successfully!
Data inserted/updated into 'TRNcustomer_journey' successfully!


C:\Users\sakth\AppData\Local\Temp\ipykernel_26172\1014440049.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result[col].fillna(empty_value, inplace=True)
C:\Users\sakth\AppData\Local\Temp\ipykernel_26172\1014440049.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

File updated successfully: C:\Users\sakth\OneDrive\Desktop\Data Science Couse\Customer Behavior Analysis\customer_reviews.csv
 Table 'TRNcustomer_reviews' created successfully!
Data inserted/updated into 'TRNcustomer_reviews' successfully!
Column 'ViewsClicksCombined' does not exist in the file.
File updated successfully: C:\Users\sakth\OneDrive\Desktop\Data Science Couse\Customer Behavior Analysis\engagement_data.csv
 Table 'TRNengagement_data' created successfully!
Data inserted/updated into 'TRNengagement_data' successfully!


In [253]:
query = """SELECT 
    YEAR(EngagementDate) AS EngagementYear, 
    CASE 
        WHEN (SUM(Clicks) >= (SELECT SUM(Clicks) / (SELECT COUNT(DISTINCT ContentType) 
                                                   FROM cust_behav_analys.TRNengagement_data) 
                              FROM cust_behav_analys.TRNengagement_data as e 
                              WHERE YEAR(e.EngagementDate) = EngagementYear) 
            AND 
              SUM(Views) >= (SELECT SUM(Views) / (SELECT COUNT(DISTINCT ContentType) 
                                                  FROM cust_behav_analys.TRNengagement_data) 
                             FROM cust_behav_analys.TRNengagement_data as e 
                             WHERE YEAR(e.EngagementDate) = EngagementYear)) 
        THEN 'Good' 
        WHEN (SUM(Clicks) >= (SELECT SUM(Clicks) / (SELECT COUNT(DISTINCT ContentType) 
                                                   FROM cust_behav_analys.TRNengagement_data) 
                              FROM cust_behav_analys.TRNengagement_data as e 
                              WHERE YEAR(e.EngagementDate) = EngagementYear) 
            OR 
              SUM(Views) >= (SELECT SUM(Views) / (SELECT COUNT(DISTINCT ContentType) 
                                                  FROM cust_behav_analys.TRNengagement_data) 
                             FROM cust_behav_analys.TRNengagement_data as e 
                             WHERE YEAR(e.EngagementDate) = EngagementYear)) 
        THEN 'Average' 
        ELSE 'Bad' 
    END AS customer_Engagement,
    ContentType,
    COUNT(ProductID) AS TotalProductsCount,COUNT(DISTINCT ProductID)  AS UniqueProductsCount,
    SUM(Clicks) AS TotalClicks,    
    SUM(Views) AS TotalViews
FROM cust_behav_analys.TRNengagement_data 
GROUP BY EngagementYear, ContentType
ORDER BY 
    EngagementYear DESC, 
    TotalProductsCount DESC, 
    TotalClicks DESC, 
    TotalViews DESC;  


"""
print('Factors are influencing customer engagement')
#factors are influencing customer engagement Based on customer reviews
fetch_data(conn,query) 


Factors are influencing customer engagement


,EngagementYear,customer_Engagement,ContentType,TotalProductsCount,UniqueProductsCount,TotalClicks,TotalViews
0,2025,Good,Video,14,8,1211,14586
1,2025,Average,Blog,9,6,527,9960
2,2025,Bad,Socialmedia,8,7,214,5465
3,2025,Bad,NEWSLETTER,6,6,609,7985
4,2024,Good,Blog,12,10,4229,24084
5,2024,Good,Video,10,10,3098,19009
6,2024,Average,newsletter,7,7,3254,17220
7,2024,Bad,SOCIALMEDIA,5,5,1491,10459
8,2023,Good,SOCIALMEDIA,11,10,10793,44012
9,2023,Good,Blog,7,7,9544,31933


In [254]:
print('Factors are influencing customer engagement')
print('No other tables are join with this table ,Only ProductID columns are match ,Not match date columns even year is not match')
data=fetch_data(conn,query) 
data.groupby(['EngagementYear', 'customer_Engagement', 'ContentType'])[['TotalProductsCount', 'UniqueProductsCount', 'TotalClicks', 'TotalViews']].sum().sort_values(by=['EngagementYear', 'TotalProductsCount', 'TotalClicks', 'TotalViews'], ascending=False) 

Factors are influencing customer engagement
No other tables are join with this table ,Only ProductID columns are match ,Not match date columns even year is not match


TotalProductsCount  \
EngagementYear customer_Engagement ContentType                       
2025           Good                Video                        14   
               Average             Blog                          9   
               Bad                 Socialmedia                   8   
                                   NEWSLETTER                    6   
2024           Good                Blog                         12   
                                   Video                        10   
               Average             newsletter                    7   
               Bad                 SOCIALMEDIA                   5   
2023           Good                SOCIALMEDIA                  11   
                                   Blog                          7   
                                   video                         7   
               Bad                 Newsletter                    4   

                                                UniqueProductsCount  \
EngagementYear customer_Engagement ContentType                        
2025           Good                Video                          8   
               Average             Blog                           6   
               Bad                 Socialmedia                    7   
                                   NEWSLETTER                     6   
2024           Good                Blog                          10   
                                   Video                         10   
               Average             newsletter                     7   
               Bad                 SOCIALMEDIA                    5   
2023           Good                SOCIALMEDIA                   10   
                                   Blog                           7   
                                   video                          6   
               Bad                 Newsletter                     4   

                                               TotalClicks TotalViews  
EngagementYear customer_Engagement ContentType                         
2025           Good                Video              1211      14586  
               Average             Blog                527       9960  
               Bad                 Socialmedia         214       5465  
                                   NEWSLETTER          609       7985  
2024           Good                Blog               4229      24084  
                                   Video              3098      19009  
               Average             newsletter         3254      17220  
               Bad                 SOCIALMEDIA        1491      10459  
2023           Good                SOCIALMEDIA       10793      44012  
                                   Blog               9544      31933  
                                   video              9494      30478  
               Bad                 Newsletter         2487      11854

In [255]:
query = """SELECT Stage, COUNT(*) AS DropOffCount
FROM cust_behav_analys.TRNcustomer_journey
WHERE Action = 'Drop-off'
GROUP BY Stage
ORDER BY DropOffCount DESC;
"""
print('At this stage are customers dropping off in their journey')
fetch_data(conn,query) 


At this stage are customers dropping off in their journey


,Stage,DropOffCount
0,Checkout,14


In [256]:
query = """SELECT 
    CASE 
        WHEN Rating =5 THEN 'Excellent'
        WHEN Rating = 4 THEN 'Good'
        WHEN Rating = 3 THEN 'Average'
        ELSE 'Poor'
    END AS RatingCategory,
    COUNT(DISTINCT CustomerID) AS UniqueCustomers,
    COUNT(DISTINCT ProductID) AS UniqueProducts,
    COUNT(ReviewID) AS TotalReviews,
    ROUND(AVG(Rating)) AS AvgRating
FROM 
    cust_behav_analys.TRNcustomer_reviews
GROUP BY 
    RatingCategory
ORDER BY 
    AvgRating desc ;
"""
print('customer reviews impact purchasing behavior')
fetch_data(conn,query) 

customer reviews impact purchasing behavior


,RatingCategory,UniqueCustomers,UniqueProducts,TotalReviews,AvgRating
0,Excellent,19,12,23,5
1,Good,34,15,39,4
2,Average,24,15,29,3
3,Poor,8,9,9,2


In [257]:
query = """
SELECT 
    j.VisitDate as Purchased_Date,
    j.Stage as Stage,
    j.Action as Action,
    j.Duration  as Duration,
    p.ProductName as Product,
    jc.CustomerName as Purchased_Customer,
    rc.CustomerName as Review_Customer,
    r.ReviewDate as Review_Date,
    r.Rating as Rating,
    r.ReviewText as Review_Text
FROM 
    cust_behav_analys.TRNcustomer_journey j     
left JOIN 
    cust_behav_analys.TRNcustomer_reviews r
ON  r.ProductID = j.ProductID and  r.ReviewDate<j.VisitDate
join cust_behav_analys.mstproducts p on j.ProductID = p.ProductID 
left join cust_behav_analys.mstcustomers jc on j.CustomerID = jc.CustomerID 
left join cust_behav_analys.mstcustomers rc on r.CustomerID = rc.CustomerID 
where j.action='Purchase'
 and  r.ReviewDate<j.VisitDate 
order by j.VisitDate desc,j.Stage,j.Action,j.Duration,Review_Date,Rating

"""
data1=fetch_data(conn,query)
data1.groupby(['Purchased_Date', 'Stage', 'Action', 'Duration', 'Product', 'Purchased_Customer','Review_Customer', 'Review_Date', 'Rating', 'Review_Text']
              ).sum().sort_values(by=['Purchased_Date', 'Review_Date'], ascending=False) 



Empty DataFrame
Columns: []
Index: [(2025-10-27, Checkout, Purchase, 246.0, Climbing Rope, Alex Rodriguez, Olivia Smith, 2025-04-21 00:00:00, 3, Good  quality,  but  could  be  cheaper.), (2025-10-27, Checkout, Purchase, 246.0, Climbing Rope, Alex Rodriguez, Michael Williams, 2025-02-02 00:00:00, 4, Exceeded  my    expectations!), (2025-10-27, Checkout, Purchase, 246.0, Climbing Rope, Alex Rodriguez, Michael Williams, 2024-05-28 00:00:00, 2, The  product  arrived    late.), (2025-10-27, Checkout, Purchase, 246.0, Climbing Rope, Alex Rodriguez, Emily Wilson, 2024-03-30 00:00:00, 5, Excellent  product,  highly  recommend!), (2025-10-27, Checkout, Purchase, 246.0, Climbing Rope, Alex Rodriguez, Alex Wilson, 2023-04-08 00:00:00, 5, Amazing  value  for  the  price.), (2025-10-27, Checkout, Purchase, 246.0, Climbing Rope, Alex Rodriguez, Sophia Miller, 2023-03-21 00:00:00, 5, Five  stars  for  the  quick  delivery.), (2025-06-15, Checkout, Purchase, 122.0, Surfboard, Chris Johnson, Jane Rodriguez, 2025-05-15 00:00:00, 3, Average  experience,  nothing  special.), (2025-06-15, Checkout, Purchase, 122.0, Surfboard, Chris Johnson, Sarah Hernandez, 2024-05-07 00:00:00, 5, Great  purchase,  very  satisfied.), (2025-06-15, Checkout, Purchase, 122.0, Surfboard, Chris Johnson, James Garcia, 2023-10-08 00:00:00, 4, Shipping  was  fast  and  the  item    was  well-packaged.), (2025-06-15, Checkout, Purchase, 122.0, Surfboard, Chris Johnson, Alex Rodriguez, 2023-05-26 00:00:00, 3, Not    worth  the  money.), (2025-06-15, Checkout, Purchase, 122.0, Surfboard, Chris Johnson, Olivia Brown, 2023-04-20 00:00:00, 5, Excellent  product,    highly  recommend!), (2025-06-15, Checkout, Purchase, 122.0, Surfboard, Chris Johnson, Michael Smith, 2023-04-14 00:00:00, 4, Five  stars  for  the  quick  delivery.), (2025-06-15, Checkout, Purchase, 122.0, Surfboard, Chris Johnson, Robert Rodriguez, 2023-03-18 00:00:00, 2, Average  experience,  nothing  special.), (2025-06-15, Checkout, Purchase, 122.0, Surfboard, Chris Johnson, James Jones, 2023-01-06 00:00:00, 5, Excellent  product,  highly  recommend!), (2025-05-29, Checkout, Purchase, 261.0, Surfboard, John Gonzalez, Jane Rodriguez, 2025-05-15 00:00:00, 3, Average  experience,  nothing  special.), (2025-05-29, Checkout, Purchase, 261.0, Surfboard, John Gonzalez, Sarah Hernandez, 2024-05-07 00:00:00, 5, Great  purchase,  very  satisfied.), (2025-05-29, Checkout, Purchase, 261.0, Surfboard, John Gonzalez, James Garcia, 2023-10-08 00:00:00, 4, Shipping  was  fast  and  the  item    was  well-packaged.), (2025-05-29, Checkout, Purchase, 261.0, Surfboard, John Gonzalez, Alex Rodriguez, 2023-05-26 00:00:00, 3, Not    worth  the  money.), (2025-05-29, Checkout, Purchase, 261.0, Surfboard, John Gonzalez, Olivia Brown, 2023-04-20 00:00:00, 5, Excellent  product,    highly  recommend!), (2025-05-29, Checkout, Purchase, 261.0, Surfboard, John Gonzalez, Michael Smith, 2023-04-14 00:00:00, 4, Five  stars  for  the  quick  delivery.), (2025-05-29, Checkout, Purchase, 261.0, Surfboard, John Gonzalez, Robert Rodriguez, 2023-03-18 00:00:00, 2, Average  experience,  nothing  special.), (2025-05-29, Checkout, Purchase, 261.0, Surfboard, John Gonzalez, James Jones, 2023-01-06 00:00:00, 5, Excellent  product,  highly  recommend!), (2025-03-09, Checkout, Purchase, 48.0, Climbing Rope, John Jones, Michael Williams, 2025-02-02 00:00:00, 4, Exceeded  my    expectations!), (2025-03-09, Checkout, Purchase, 48.0, Climbing Rope, John Jones, Michael Williams, 2024-05-28 00:00:00, 2, The  product  arrived    late.), (2025-03-09, Checkout, Purchase, 48.0, Climbing Rope, John Jones, Emily Wilson, 2024-03-30 00:00:00, 5, Excellent  product,  highly  recommend!), (2025-03-09, Checkout, Purchase, 48.0, Climbing Rope, John Jones, Alex Wilson, 2023-04-08 00:00:00, 5, Amazing  value  for  the  price.), (2025-03-09, Checkout, Purchase, 48.0, Climbing Rope, John Jones, Sophia Miller, 2023-03-21 00:00:00, 5, Five  stars  for  the  quick  delivery.), (

In [258]:
query = """
SELECT p.ProductName ,g.Country,g.City,c.CustomerName,j.Stage,j.Action,sum(j.Duration) 
FROM cust_behav_analys.trncustomer_journey j
join cust_behav_analys.mstcustomers c on j.CustomerID = c.CustomerID  
join cust_behav_analys.mstproducts p on j.ProductID = p.ProductID 
join cust_behav_analys.mstgeography g on g.GeographyID = c.GeographyID 
where  action='Purchase'
group by j.ProductID ,g.GeographyID,g.City,j.CustomerID,j.Stage,j.Action
order by j.ProductID ,g.GeographyID,g.City,j.CustomerID,j.Stage,j.Action,sum(j.Duration)
"""
print("Which products, locations, and customer segments are performing well?")
fetch_data(conn,query)

Which products, locations, and customer segments are performing well?


,ProductName,Country,City,CustomerName,Stage,Action,sum(j.Duration)
0,Fitness Tracker,Germany,Berlin,Daniel Thomas,Checkout,Purchase,137.0
1,Climbing Rope,Netherlands,Amsterdam,John Jones,Checkout,Purchase,48.0
2,Climbing Rope,Sweden,Stockholm,Alex Rodriguez,Checkout,Purchase,246.0
3,Kayak,Austria,Vienna,Jane Williams,Checkout,Purchase,87.0
4,Surfboard,UK,London,John Gonzalez,Checkout,Purchase,261.0
5,Surfboard,Spain,Madrid,Chris Johnson,Checkout,Purchase,122.0


In [259]:
query = """SELECT ReviewText as Sentiment FROM cust_behav_analys.trncustomer_reviews;"""
df_reviews = fetch_data(conn,query)
# Apply sentiment function
df_reviews["Sentiment"] = df_reviews["Sentiment"].apply(get_sentiment)

# Group and count sentiments
df_reviews.groupby("Sentiment").size().reset_index(name='No_Of_Sentiment')

,Sentiment,No_Of_Sentiment
0,Negative,11
1,Neutral,5
2,Positive,84


In [260]:
query = """SELECT CustomerName,ReviewText as sentiment FROM cust_behav_analys.trncustomer_reviews r
join cust_behav_analys.mstcustomers c on r.CustomerID = c.CustomerID;
"""
data2 = fetch_data(conn,query)
data2["sentiment"] = data2["sentiment"].apply(get_sentiment)
data2=data2.groupby(['sentiment','CustomerName']
              ).size().reset_index(name='No_Of_Sentiment').query('No_Of_Sentiment > 1').sort_values(by=['sentiment','CustomerName' ])
data2.groupby(['sentiment','CustomerName'] ).sum('No_Of_Sentiment')         

No_Of_Sentiment
sentiment CustomerName                     
Negative  Daniel Miller                   2
Positive  Alex Wilson                     2
          Daniel Thomas                   2
          David Lopez                     2
          David Thomas                    2
          Emily Lopez                     2
          Emily Wilson                    2
          Emma Wilson                     2
          Jane Rodriguez                  5
          John Garcia                     3
          John Rodriguez                  2
          Michael Gonzalez                2
          Michael Lopez                   2
          Michael Martinez                2
          Olivia Miller                   2
          Olivia Smith                    5
          Robert Hernandez                3
          Sarah Martinez                  2
          Sophia Anderson                 2

In [261]:
query = """
SELECT 'Delivery issue' as categories,
    LOWER(ReviewText) AS review_lower 
    from cust_behav_analys.trncustomer_reviews
WHERE 
    LOWER(ReviewText) LIKE '%late%' 
    OR LOWER(ReviewText) LIKE '%broken%' 
    OR LOWER(ReviewText) LIKE '%miss%'   
UNION ALL
SELECT 'Quality issue' as categories,
    LOWER(ReviewText) AS review_lower 
    from cust_behav_analys.trncustomer_reviews
WHERE  LOWER(ReviewText) LIKE '%poor%' 
    OR LOWER(ReviewText) LIKE '%bad%' 
    OR LOWER(ReviewText) LIKE '%worst%' 
    OR LOWER(ReviewText) LIKE '%slow%' 
    OR( LOWER(ReviewText) LIKE '%not%' and LOWER(ReviewText) LIKE '%worth%' )
    OR( LOWER(ReviewText) LIKE '%not%' and LOWER(ReviewText) LIKE '%working%' )
    OR( LOWER(ReviewText) LIKE '%not%' and LOWER(ReviewText) LIKE '%good%' )
    OR LOWER(ReviewText) LIKE '%issue%'
    order by categories, review_lower;
    
 
"""
print("Identify key complaints and improvement areas.")
fetch_data(conn,query)

Identify key complaints and improvement areas.


,categories,review_lower
0,Delivery issue,the product arrived late.
1,Quality issue,i had a bad experience with this product.
2,Quality issue,not worth the money.
3,Quality issue,not worth the money.
4,Quality issue,not worth the money.
5,Quality issue,not worth the money.
6,Quality issue,not worth the money.
7,Quality issue,not worth the money.
8,Quality issue,not worth the money.
9,Quality issue,not worth the money.


In [262]:
query = """
SELECT 
    p.ProductName,
    j.Action,
    COUNT(*) AS NegativeReviewCount
FROM 
    cust_behav_analys.trncustomer_reviews r
JOIN 
    cust_behav_analys.mstproducts p ON r.ProductID = p.ProductID 
JOIN
    cust_behav_analys.trncustomer_journey j ON r.ProductID = j.ProductID and r.ReviewDate<j.VisitDate
WHERE 
    LOWER(ReviewText) LIKE '%late%' 
    OR LOWER(ReviewText) LIKE '%broken%' 
    OR LOWER(ReviewText) LIKE '%miss%'
    or  LOWER(ReviewText) LIKE '%poor%' 
    OR LOWER(ReviewText) LIKE '%bad%' 
    OR LOWER(ReviewText) LIKE '%worst%' 
    OR LOWER(ReviewText) LIKE '%slow%' 
    OR( LOWER(ReviewText) LIKE '%not%' and LOWER(ReviewText) LIKE '%worth%' )
    OR( LOWER(ReviewText) LIKE '%not%' and LOWER(ReviewText) LIKE '%working%' )
    OR( LOWER(ReviewText) LIKE '%not%' and LOWER(ReviewText) LIKE '%good%' )
    OR LOWER(ReviewText) LIKE '%issue%'
GROUP BY 
    p.ProductID, j.Action
ORDER BY 
    NegativeReviewCount DESC;

"""
print("patterns between negative reviews and product performance.")
fetch_data(conn, query)

patterns between negative reviews and product performance.


,ProductName,Action,NegativeReviewCount
0,Kayak,View,6
1,Swim Goggles,View,2
2,Climbing Rope,View,2
3,Surfboard,View,2
4,Tennis Racket,View,2
5,Surfboard,Purchase,2
6,Climbing Rope,Purchase,2
7,Kayak,Purchase,2
8,Fitness Tracker,View,1
9,Tennis Racket,Drop-off,1


In [263]:
query = """WITH categorized_reviews AS (
    SELECT 'Delivery issue' AS categories, 1 AS categories_count
    FROM cust_behav_analys.trncustomer_reviews
    WHERE 
        LOWER(ReviewText) LIKE '%late%' 
        OR LOWER(ReviewText) LIKE '%broken%' 
        OR LOWER(ReviewText) LIKE '%miss%'   

    UNION ALL

    SELECT 'Quality issue' AS categories, 1 AS categories_count
    FROM cust_behav_analys.trncustomer_reviews
    WHERE  
        LOWER(ReviewText) LIKE '%poor%' 
        OR LOWER(ReviewText) LIKE '%bad%' 
        OR LOWER(ReviewText) LIKE '%worst%' 
        OR LOWER(ReviewText) LIKE '%slow%' 
        OR (LOWER(ReviewText) LIKE '%not%' AND LOWER(ReviewText) LIKE '%worth%')
        OR (LOWER(ReviewText) LIKE '%not%' AND LOWER(ReviewText) LIKE '%working%')
        OR (LOWER(ReviewText) LIKE '%not%' AND LOWER(ReviewText) LIKE '%good%')
        OR LOWER(ReviewText) LIKE '%issue%'
)

SELECT categories, SUM(categories_count) AS total_count
FROM categorized_reviews
GROUP BY categories
ORDER BY total_count DESC
LIMIT 1;
"""
print("Recommend strategies to enhance customer satisfaction.")
data4=fetch_data(conn, query)

print("Recommend: '" + data4['categories'][0] + "' is coming repeatedly (" + str(data4['total_count'][0]) + " times), so if we solve this, it will enhance customer satisfaction.")



Recommend strategies to enhance customer satisfaction.
Recommend: 'Quality issue' is coming repeatedly (9 times), so if we solve this, it will enhance customer satisfaction.
